In [1]:
import gym
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import scipy.linalg as la

# Model Free Stabilization and Optimization
Here we will just demonstrate a "model-free" procedure (which does not compute $(A,B,Q,R)$). It is assumed that there are $n$ states and $p$ inputs.

Say that $Q$ and $R$ are both positive definite. Then if the system is stabilizable, it computes a sequence of gains $K_i$  with the following properties:

* $K_i$ is a stabilizing controller for all sufficiently large $i$.
* The algorithm provides a certificate of stability  once a stabilizing controller is found.
* $K_i$ converges to the optimum gain geometrically. 
    
    
The method works as follows:
* Generate $N$ samples $(x_j,u_j,c_j,x_j^+)$, where
    - $x_j^+=Ax_j+Bu_j$.
    - $c_j = x_j^\top Q x_j + u_j^\top R u_j$
    - Any $u_j$ sequence with sufficient noise injection for invertibility will work. 
    - Having $N$ of order $(n+p)^2$ works.
* Start with $K_i=0$ and discount factor $\gamma_i = 0$.
* For each $i\ge 0$ do the following.
    - Solve for the action-value function, $Q_{i}(x,u)$ by least-squares, based on the system of equations $c_j = Q_{i}(x_j,u_j) - \gamma_i Q_{i}(x_j^+,K_i x_j^+)$. 
    - Compute $P_i$ such that $x^\top P_i x = Q_i(x,K_i x)$. This is the matrix corresponding to the value function.
    - Let $\gamma_{i+1} > \gamma_i$  be chosen as follows:
        - Let $\hat Q_i$ be the action-value function computed from the equations $c_j = \hat Q_{i}(x_j,u_j) - \gamma_{i+1} \hat Q_{i}(x_j^+,K_i x_j^+)$
        - Let $\hat P_i$ be the corresponding value function matrix
        - Use bisection to find the largest value of $\gamma_{i+1}$ so that
            - $\gamma_{i+1} \le 1$
            - $\hat P_i$ is positive definite
            - $\|P_i - \hat P_i \|\le 1$. (For some matrix norm, e.g. Frobenius or induced 2 norm.)
           
    - Let $K_{i+1}$ be the minimizing gain: $\min_u \hat Q_i(x,u) = \hat Q_i(x,K_{i+1})$
    
    
With this description, the properties of the algorithm can be described more precisely.
* $\gamma_i$ is monotonically increasing.
* For all $i$, $K_i$ stabilizes the system $(\sqrt{\gamma_i} A,\sqrt{\gamma_i}B)$.
* If $\gamma_i = 1$, then $K_i$ is stabilizing. Furthermore, $\gamma_j=1$ and $K_j$ is stabilizing for all $j\ge i$.
* If the system is stabilizable, then:
    - $\gamma_i=1$ after finitely many steps
    - Once $\gamma_i=1$, the steps become equivalent to policy iteration, and thus converge to the optimal gain.
* If the system is not stabilizable,then $\lim_{i\to\infty} \gamma_i =\hat\gamma$ such that $(\sqrt{\hat \gamma} A,\sqrt{\hat \gamma }B)$ is not stabilizable. 

In [2]:
#######  Helper Code #######

# Use this to generate random systems for testing
import numpy.random as rnd
import scipy.stats as st

def randomSystem():

    n = rnd.randint(1,10)
    p = rnd.randint(1,10)

    A = rnd.randn(n,n)
    B = rnd.randn(n,p)

    Q = st.wishart.rvs(n+1,np.eye(n)).reshape((n,n))
    R = st.wishart.rvs(p+1,np.eye(p)).reshape((p,p))
    
    return A,B,Q,R

def trilStack(M):
    n = len(M)
    v = []
    for i in range(n):
        v.append(M[i:,i])
    return np.hstack(v)

def trilUnstack(v):
    d = len(v)
    n = int((np.sqrt(1+8*d)-1)/2)
    M = np.zeros((n,n))
    j = 0
    for i in range(n):
        M[i:,i] = v[j:j+n-i]
        j = j+n-i
    return M

def symFromTrilStack(v):
    L = trilUnstack(v)
    D = np.diag(np.diag(L))
    return L + L.T - D

def quadraticMonomials(z):
    M = np.outer(z,z)
    M_L = np.tril(M,-1)
    M_D = np.diag(np.diag(M))
    return trilStack(2*M_L + M_D)

def buildPhi(X,U,X_next,K,gamma):
    Horizon = len(U)
    Phi = []
    for x,u,x_next in zip(X,U,X_next):
        z = np.hstack([x,u])
        z_next = np.hstack([x_next,K@x_next])
        # Accounting for noise requires extra parameter
        psi = np.hstack(quadraticMonomials(z))
        psi_next = np.hstack(quadraticMonomials(z_next))
        phi = psi - gamma * psi_next
        Phi.append(phi)
    return np.array(Phi)
    
    
    
def getVandQmatrices(X,U,X_next,c,K,gamma):
    F = np.vstack([np.eye(n),K])
    Phi = buildPhi(X,U,X_next,K,gamma)
    theta = la.lstsq(Phi,c)[0]
    # The first entry of theta is the noise variance
    Q_mat = symFromTrilStack(theta)
    P = F.T @ Q_mat @ F
    return P,Q_mat
    
def increaseGamma(X,U,X_next,c,K,gamma):
    LB = gamma 
    UB = 1.
    
    p,n = K.shape
    
    
    
    minEig = -np.inf
    while minEig < 0:
        gamma = .5 * (LB + UB)
        P,Q_mat = getVandQmatrices(X,U,X_next,c,K,gamma)
        minEig = la.eigvalsh(P,eigvals=[0,0])[0]
        if minEig < 0:
            UB = gamma
    return gamma, Q_mat, P
    
def genX(n,b):
    r = rnd.rand() * b
    x = rnd.randn(n)
    return r * x / la.norm(x)

#### Main Code ##### 

A,B,Q,R = randomSystem()
n,p = B.shape

NumEpisodes = 10
x_bound = 100
x = genX(n,x_bound)

Horizon = (p+n)*(p+n+1)
K = np.zeros((p,n))
gamma = 0.


P_opt = la.solve_discrete_are(A,B,Q,R)
K_opt = -la.solve(R+B.T@P_opt@B,B.T@P_opt@A)

K_err = 10

ep =0

X = []
U = []
X_next = []
c = []# Simulate the system
for t in range(Horizon):
    u = K@x + .1 * rnd.randn(p)
    x_next = A@x + B @ u 
    c.append(x@Q@x + u@R@u)
    X.append(x)
    U.append(u)
    X_next.append(x_next)
    if la.norm(x_next) <= x_bound:
        x = x_next
    else:
            x = genX(n,x_bound)
        
c = np.array(c)

while K_err > 1e-6 :
    ep += 1
 
    
    eigMax = np.max(np.abs(la.eigvals(A+B@K)))
    K_err = la.norm(K-K_opt)
    print('Episode: ',ep,', ||K-K*||:',K_err, ', rho(A+BK):', eigMax)
    gamma,Q_mat, P = increaseGamma(X,U,X_next,c,K,gamma)
    # improve K
    Omega = Q_mat[n:,n:]
    Psi = Q_mat[n:,:n]
    K = -la.solve(Omega,Psi)
    

Episode:  1 , ||K-K*||: 3.868315691617771 , rho(A+BK): 1.9040184794037724
Episode:  2 , ||K-K*||: 2.8766882218309835 , rho(A+BK): 1.6723047176082888
Episode:  3 , ||K-K*||: 0.9308276830585793 , rho(A+BK): 1.1148148625498264
Episode:  4 , ||K-K*||: 0.9499224063043128 , rho(A+BK): 0.6396130225834656
Episode:  5 , ||K-K*||: 0.31625599430069024 , rho(A+BK): 0.6170621172092513
Episode:  6 , ||K-K*||: 0.09788448532334008 , rho(A+BK): 0.5180597822740226
Episode:  7 , ||K-K*||: 0.05266532345318559 , rho(A+BK): 0.5034147441102927
Episode:  8 , ||K-K*||: 0.025924941141055445 , rho(A+BK): 0.4938285390517079
Episode:  9 , ||K-K*||: 0.012903803770933208 , rho(A+BK): 0.4891670938652407
Episode:  10 , ||K-K*||: 0.006436231726987612 , rho(A+BK): 0.48685499074585764
Episode:  11 , ||K-K*||: 0.0032141683506543865 , rho(A+BK): 0.4857037382491112
Episode:  12 , ||K-K*||: 0.0016060898508915267 , rho(A+BK): 0.4851293172879408
Episode:  13 , ||K-K*||: 0.0008027982624560703 , rho(A+BK): 0.48484237254623114
Ep